# EM for Binary Data

In [11]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, linewidth=100)

In [2]:
dataset = np.loadtxt('binarydigits.txt')
N, D = dataset.shape

N, D

(100, 64)

We have $N = 100$ images, each with $D = 64$ pixels laid out as a vector, i.e., $\mathbf{x}^{\left(n\right)} \in \mathbb{R}^D,\ \forall n\in \left\{1,\ldots,N\right\}$. 

## 3 (d)

In [91]:
class EMForMultivariateBernoulli:

    def log_likelihood(self, X, parameters, pi):

        odds = np.divide(np.minimum(parameters, 1-1e-10), 1-np.minimum(parameters, 1-1e-10))
        marginals = list()
        for x in X:
            likelihoods = np.prod(np.multiply(1-parameters, np.power(odds, x)), axis=1)
            joints = np.multiply(pi, likelihoods)
            marginal = np.sum(joints)
            marginals.append(marginal)
        marginals = np.array(marginals)
        log_likelihood = np.sum(np.log(marginals))

        return log_likelihood

    def expectation(self, X, parameters, priors):

        odds = np.divide(np.minimum(parameters, 1-1e-10), 1-np.minimum(parameters, 1-1e-10))
        responsibilities = list()
        for x in X:
            likelihoods = np.prod(np.multiply(1-parameters, np.power(odds, x)), axis=1)
            joints = np.multiply(priors, likelihoods)
            marginal = np.sum(joints)
            posteriors = joints / marginal
            responsibilities.append(posteriors)
        responsibilities = np.array(responsibilities)
        
        return responsibilities

    def maximisation(self, X, responsibilities):

        P = np.divide(responsibilities.T @ X, np.sum(responsibilities, axis=0).reshape(-1, 1))
        pi = np.mean(responsibilities, axis=0)

        return P, pi

    def __call__(self, K, X, n_iterations, epsilon):

        P = np.random.uniform(size=(K, X.shape[1]))
        pi = np.ones(K) / K

        log_ls = [self.log_likelihood(X, P, pi)]

        for i in range(n_iterations):
            responsibilities = self.expectation(X=X, parameters=P, priors=pi)
            P, pi = self.maximisation(X, responsibilities=responsibilities)
            log_l = self.log_likelihood(X, parameters=P, pi=pi)
            log_ls.append(log_l)
            if log_ls[-1] - log_ls[-2] <= epsilon:
                break

        return P, pi, responsibilities, log_ls

In [45]:
def smoothing(p, eps=1e-8):

    nonzero = np.nonzero(p)
    if nonzero[0].size == 0:
        return p
    p[nonzero] -= eps * p.size/nonzero[0].size
    p += eps

    return p


class EMForMultivariateBernoulli:

    def log_likelihood(self, X, P, pi):

        N, D, K = *X.shape, pi.size

        log_likelihood = 0
        for n in range(N):
            x = X[n]
            x_likelihood = 0
            for k in range(K):
                component_joint = pi[k]
                for d in range(D):
                    component_joint *= (P[k, d]**X[n, d] * (1-P[k, d])**(1-X[n, d]))
                x_likelihood += component_joint
            log_likelihood += np.log(x_likelihood)

        return log_likelihood

    def expectation(self, X, parameters, priors):

        parameters = 1 - smoothing(1-parameters)
        odds = np.divide(parameters, 1-parameters)
        responsibilities = list()
        for x in X:
            likelihoods = np.prod(np.multiply(1-parameters, np.power(odds, x)), axis=1)
            joints = np.multiply(priors, likelihoods)
            marginal = np.sum(joints)
            posteriors = joints / marginal
            responsibilities.append(posteriors)
        responsibilities = np.array(responsibilities)
        
        return responsibilities

    def maximisation(self, X, responsibilities):

        P = np.divide(responsibilities.T @ X, smoothing(np.sum(responsibilities, axis=0)).reshape(-1, 1))
        pi = np.mean(responsibilities, axis=0)

        return P, pi

    def __call__(self, K, X, n_iterations, epsilon):

        P = np.random.uniform(size=(K, X.shape[1]))
        pi = np.ones(K) / K

        log_ls = [self.log_likelihood(X, P, pi)]

        for i in tqdm(range(n_iterations)):
            responsibilities = self.expectation(X=X, parameters=P, priors=pi)
            P, pi = self.maximisation(X, responsibilities=responsibilities)
            log_l = self.log_likelihood(X, P=P, pi=pi)
            log_ls.append(log_l)
            if log_ls[-1] - log_ls[-2] <= epsilon:
                break

        return P, pi, responsibilities, log_ls

In [52]:
def smoothing(p, eps=1e-8):

    nonzero = np.nonzero(p)
    if nonzero[0].size == 0:
        return p
    p[nonzero] -= eps * p.size/nonzero[0].size
    p += eps

    return p


class EMForMultivariateBernoulli:

    def log_likelihood(self, X, P, pi):

        N, D, K = *X.shape, pi.size

        log_likelihood = 0
        for n in range(N):
            x = X[n]
            x_likelihood = 0
            for k in range(K):
                component_joint = pi[k]
                for d in range(D):
                    component_joint *= (P[k, d]**X[n, d] * (1-P[k, d])**(1-X[n, d]))
                x_likelihood += component_joint
            log_likelihood += np.log(x_likelihood)

        return log_likelihood

    def expectation(self, X, parameters, priors):

        N, D, K = *X.shape, priors.size

        parameters = 1 - smoothing(1-parameters)
        responsibilities = np.zeros(shape=(N, K))
        for n in range(N):
            for k in range(K):
                responsibilities[n, k] = priors[k]
                for d in range(D):
                    responsibilities[n, k] *= (parameters[k, d]**X[n, d] * (1-parameters[k, d])**(1-X[n, d]))
            responsibilities[n] /= np.sum(responsibilities[n])
        
        return responsibilities

    def maximisation(self, X, responsibilities):

        P = np.divide(responsibilities.T @ X, smoothing(np.sum(responsibilities, axis=0)).reshape(-1, 1))
        pi = np.mean(responsibilities, axis=0)

        return P, pi

    def __call__(self, K, X, n_iterations, epsilon):

        P = np.random.uniform(size=(K, X.shape[1]))
        pi = np.ones(K) / K

        log_ls = [self.log_likelihood(X, P, pi)]

        for i in tqdm(range(n_iterations)):
            responsibilities = self.expectation(X=X, parameters=P, priors=pi)
            P, pi = self.maximisation(X, responsibilities=responsibilities)
            log_l = self.log_likelihood(X, P=P, pi=pi)
            log_ls.append(log_l)
            if log_ls[-1] - log_ls[-2] <= epsilon:
                break

        return P, pi, responsibilities, log_ls

In [65]:
em = EMForMultivariateBernoulli()
P, pi, responsibilities, log_ls = em(K=3, X=dataset[:4, 48:53], n_iterations=1000, epsilon=-1e-10)

  0%|          | 2/1000 [00:00<00:02, 481.38it/s]
